# Creación de sistemas RAG sobre bases de datos vectoriales

Este proyecto implementa un sistema de Recuperación-Augmentación-Generación (RAG) que construye un almacén de vectores (vector store) a partir de datos extraídos de una página web. Utiliza bibliotecas como `langchain`, `requests` y `beautifulsoup4` para procesar el contenido.

## Instalación de dependencias

Primero, instala las bibliotecas necesarias para el proyecto:

```python
# Instalar las bibliotecas necesarias:
!pip install langchain requests beautifulsoup4 langchain_huggingface chromadb


# Extracción del contenido de una página web
Este bloque de código se encarga de obtener el contenido de una página web utilizando requests para la solicitud HTTP y BeautifulSoup para el análisis HTML.

In [1]:
# Extracción de contenido de la página de Wikipedia:
# Usaremos requests para obtener el HTML de la página y BeautifulSoup
import requests
from bs4 import BeautifulSoup
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()


# División del texto en fragmentos
Para manejar grandes cantidades de texto, dividimos el contenido en fragmentos manejables. Esto facilita el procesamiento posterior.

In [2]:
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


# Configuración del modelo LLM
Aquí se define el modelo de lenguaje (LLM) que se utilizará para las consultas. En este caso, se usa OllamaLLM.

In [3]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")


# Creación del modelo de embeddings y del almacén de vectores
El modelo de embeddings convierte texto en representaciones vectoriales, que luego se almacenan en Chroma.

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)


/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento del contenido de la página web
Se obtiene el contenido de la página web y se divide en fragmentos manejables.

In [5]:
url = "https://english.elpais.com/usa/2024-12-16/trump-says-hell-deport-criminal-migrants-first-but-who-is-he-referring-to.html"
page_content = extract_text_from_url(url) 

chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 29


# Adición de documentos al almacén de vectores
Cada fragmento de texto se transforma en un documento que se agrega al almacén de vectores.

In [6]:
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")


Documents added to the vector store.


# Creación del prompt para el sistema RAG
Se define un prompt para guiar el modelo en la generación de respuestas basadas en el contexto recuperado.

In [7]:
prompt = ChatPromptTemplate.from_template(
    template="Use the context below to answer the user's question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
)


# Configuración del sistema RAG
El sistema utiliza el almacén de vectores como recuperador para construir una cadena de consulta y generación.

In [8]:
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


# Consulta al sistema RAG
Por último, se realiza una consulta al sistema para obtener una respuesta basada en el contenido de la página web.

In [9]:
query = "Does it talk about Trump?"
response = qa_chain.invoke(query)
print("Response:", response)


Response: {'query': 'Does it talk about Trump?', 'result': 'Yes, the article talks about Trump. It specifically mentions:\n\n* Donald Trump as the president-elect who claimed that 13,000 undocumented murderers are roaming the country.\n* Trump\'s first term when his administration enforced the "zero tolerance" policy at the border.\n* Trump\'s deportation policies, which focused more on migrants who crossed the border illegally rather than those convicted of crimes.\n* Trump\'s current stance on immigration and his mass deportation plan.'}
